## Baseline Estimation

In [1]:
import pandas as pd
import numpy as np
from tick_loss import *
import requests

In [2]:
# url = "https://raw.githubusercontent.com/TobiPfeiffersGitHub/BSExNovartis_Thesis/main/Data/monthly_data.csv"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("monthly_data.csv", "wb") as file:
#         file.write(response.content)
#     print("File downloaded successfully.")
# else:
#     print("Failed to download the file.")

# Load the file into a DataFrame
# data = pd.read_csv("monthly_data.csv")
data = pd.read_csv('clean_monthly_data.csv')

data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


We cut off the first observations and the last 5 to ensure the same amount of data.

In [3]:
# data = data.drop(data.index[0])
# data = data.drop(data.index[-5:])
# data = data.reset_index(drop=True)
# data = data.drop(columns=['Unnamed: 0'])
# data['Date'] = pd.to_datetime(data['Date'])
# data

1. 1 month 

In [4]:
def baseline_performance(data, ticker, alpha, period):
    """
    arguements: DataFrame, Name of the Ticker, alpha (level of risk), period e.g. 12 (month)
    
    the function calculates the historical quantile and the tickloss for this historical quantile 
    for every month and averages it over the length of the hold out sample
    """
    cutoff = period
    sample_length = len(data)-cutoff
    hold_out = data.tail(cutoff)
    hold_out = hold_out.reset_index(drop=True)

    loss = 0

    for i in range(0, cutoff):
        sample = data[i:sample_length+i]
        var = get_historical_quantiles(sample, [str(ticker)], alpha)
        r = hold_out[str(ticker)][i]
        loss += tick_loss(alpha, r, var)

    loss = loss/period
    return loss

In [6]:
nvs_m_loss = baseline_performance(data, 'NVS', 0.05, 12)
print(f'NVS: {nvs_m_loss}')

# abbv_m_loss = baseline_performance(data, 'ABBV', 0.05, 12)
# print(f'ABBV: {abbv_m_loss}')

azn_m_loss = baseline_performance(data, 'AZN', 0.05, 12)
print(f'AZN: {azn_m_loss}')

bmy_m_loss = baseline_performance(data, 'BMY', 0.05, 12)
print(f'BMY: {bmy_m_loss}')

jnj_m_loss = baseline_performance(data, 'JNJ', 0.05, 12)
print(f'JNJ: {jnj_m_loss}')

lly_m_loss = baseline_performance(data, 'LLY', 0.05, 12)
print(f'LLY: {lly_m_loss}')

mrk_m_loss = baseline_performance(data, 'MRK', 0.05, 12)
print(f'MRK: {mrk_m_loss}')

nvo_m_loss = baseline_performance(data, 'NVO', 0.05, 12)
print(f'NVO: {nvo_m_loss}')

pfe_m_loss = baseline_performance(data, 'PFE', 0.05, 12)
print(f'PFE: {pfe_m_loss}')

rog_m_loss = baseline_performance(data, 'ROG', 0.05, 12)
print(f'ROG: {rog_m_loss}')

NVS: 0.4770840902535467
AZN: 0.7399933085282727
BMY: 0.5483543888261865
JNJ: 0.4930074529044592
LLY: 0.6527370268249068
MRK: 0.6716556625292887
NVO: 0.6147867657764806
PFE: 0.827537276633516
ROG: 3.8156624530976817


2. 3 months

In [7]:
def baseline_performance_3m(data, ticker, alpha, dates):    
    t_loss = 0
    dta = data.copy()

    for d in dates:
        split_date = d
        train_df = dta[dta.Date <= split_date]
        test_df = dta[dta.Date > split_date]
        test_df = test_df.reset_index(drop=True)
        hq = train_df[str(ticker)].quantile(alpha)
        var = pd.Series([hq]*3)
        r = test_df[str(ticker)][0:3]
        tickloss = tick_loss(alpha, r, var)
    
        t_loss += tickloss
        
    t_loss = t_loss/10
    return t_loss

In [8]:
dates3m = ['2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [10]:
tickers = ['NVS', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG']

for t in tickers:
    loss = baseline_performance_3m(data, t, 0.05, dates3m)
    print(f'{t}: {loss}')

NVS: 1.299420643390336
AZN: 2.1639432739971474
BMY: 1.873368787020827
JNJ: 1.4425363868004957
LLY: 1.8069039381560068
MRK: 2.2152677219952226
NVO: 1.7302032919156694
PFE: 2.3942485933219437
ROG: 12.951047146117011


3. 6 months

In [11]:
def baseline_performance_6m(data, ticker, alpha, dates):
    
    t_loss = 0
    dta = data.copy()

    for d in dates:
        split_date = d
        train_df = dta[dta.Date <= split_date]
        test_df = dta[dta.Date > split_date]
        test_df = test_df.reset_index(drop=True)
        hq = train_df[str(ticker)].quantile(alpha)
        var = pd.Series([hq]*6)
        r = test_df[str(ticker)][0:6]
        tickloss = tick_loss(alpha, r, var)
    
        t_loss += tickloss
        
    t_loss = t_loss/7
    return t_loss

In [12]:
dates6m  = ['2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [13]:
for t in tickers:
    loss = baseline_performance_6m(data, t, 0.05, dates6m)
    print(f'{t}: {loss}')

NVS: 2.376176484833636
AZN: 4.600804616766299
BMY: 3.790483127451925
JNJ: 3.0785149656210273
LLY: 3.610177297048033
MRK: 4.3789045021515465
NVO: 3.2237337945550784
PFE: 4.274211277147958
ROG: 24.60565683007282


In [14]:
for t in tickers:
    loss = baseline_performance_6m(data, t, 0.05, dates6m)
    print(f'{t}: {loss}')

NVS: 2.376176484833636
AZN: 4.600804616766299
BMY: 3.790483127451925
JNJ: 3.0785149656210273
LLY: 3.610177297048033
MRK: 4.3789045021515465
NVO: 3.2237337945550784
PFE: 4.274211277147958
ROG: 24.60565683007282


4. 9 months

In [15]:
def baseline_performance_9m(data, ticker, alpha, dates):
    
    t_loss = 0
    dta = data.copy()

    for d in dates:
        split_date = d
        train_df = dta[dta.Date <= split_date]
        test_df = dta[dta.Date > split_date]
        test_df = test_df.reset_index(drop=True)
        hq = train_df[str(ticker)].quantile(alpha)
        var = pd.Series([hq]*9)
        r = test_df[str(ticker)][0:9]
        tickloss = tick_loss(alpha, r, var)
    
        t_loss += tickloss
        
    t_loss = t_loss/4
    return t_loss

In [16]:
dates9m = ['2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01']

In [17]:
for t in tickers:
    loss = baseline_performance_9m(data, t, 0.05, dates9m)
    print(f'{t}: {loss}')

NVS: 3.8686992993292373
AZN: 6.580494084890573
BMY: 5.7376516385038085
JNJ: 4.4695713752356205
LLY: 5.457769393548885
MRK: 6.672315525554376
NVO: 5.168817429508104
PFE: 6.827463147415223
ROG: 42.07295561194244


5. 12 months

In [18]:
def tick_loss2(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)
    print(df)
    t_loss = 0

    for i in df.index:
        t_loss += (alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i]) + 
                 (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i])

    return t_loss

In [19]:
hold_out = data.tail(12)
hold_out = hold_out.reset_index(drop=True)
sample_length = len(data) - 12
sample = data.head(sample_length)

In [20]:
for t in tickers:
    hq = sample[t].quantile(0.05)
    var = pd.Series([hq]*12)
    r = hold_out[t]
    loss = tick_loss(0.05, r, var)
    print(f'{t}: {loss}')

NVS: 5.864406640609334
AZN: 8.437524308351755
BMY: 6.820382519025058
JNJ: 5.545295420404286
LLY: 7.620263336613559
MRK: 8.101276325860175
NVO: 7.3853357957944334
PFE: 9.34322943985131
ROG: 45.755904809487255
